In [154]:
import config
from bs4 import BeautifulSoup as BS
import requests
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import *
import time
import re
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV

from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df1 = pd.read_csv('understat1.csv', index_col=0)

In [6]:
df1 = df1.reset_index()

In [10]:
df1['index'].value_counts()

EPL           100
La_liga       100
Ligue_1       100
Serie_A       100
Bundesliga     90
RFPL           80
Name: index, dtype: int64

In [14]:
conditionR = df1['index'] != 'RFPL'

In [15]:
df1 = df1[conditionR]

In [17]:
df1['index'].value_counts()

EPL           100
La_liga       100
Ligue_1       100
Serie_A       100
Bundesliga     90
Name: index, dtype: int64

In [20]:
df1.columns

Index(['index', 'Unnamed: 1', 'position', 'team', 'matches', 'wins', 'draws',
       'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA',
       'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep',
       'deep_allowed', 'xpts', 'xpts_diff'],
      dtype='object')

In [21]:
col_list = ['league', 'year', 'position', 'team', 'matches', 'wins', 'draws',
       'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA',
       'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep',
       'deep_allowed', 'xpts', 'xpts_diff']

In [22]:
df1.columns = col_list

In [298]:
conditionY = df1['year'] == 2018

In [305]:
df_stats = df1[conditionY]

In [306]:
df_stats.head()

,league,year,position,team,matches,wins,draws,loses,scored,missed,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
72,Bundesliga,2018,1,Bayern Munich,34,24,6,4,88,32,...,27.412344,-4.587656,22.865761,66.344146,8.642573,30.236541,435,89,82.0030,4.0030
73,Bundesliga,2018,2,Borussia Dortmund,34,23,7,4,81,44,...,42.875457,-1.124543,39.086641,22.876651,14.560878,15.574332,268,159,62.8540,-13.1460
74,Bundesliga,2018,3,RasenBallsport Leipzig,34,19,9,6,63,29,...,39.653422,10.653422,35.106988,27.835156,10.663369,10.547469,245,160,63.0567,-2.9433
75,Bundesliga,2018,4,Bayer Leverkusen,34,18,4,12,69,52,...,45.710428,-6.289572,41.163765,16.537340,10.264006,15.042459,288,165,57.4020,-0.5980
76,Bundesliga,2018,5,Borussia M.Gladbach,34,16,7,11,55,42,...,52.868345,10.868345,50.595009,-1.075537,16.613170,18.875894,237,227,51.3782,-3.6218


In [28]:
df_epl1 = pd.read_csv('epl1.csv')

In [30]:
df_seriea1 = pd.read_csv('seriea1.csv')

In [31]:
df_laliga1 = pd.read_csv('laliga1.csv')

In [32]:
df_bundesliga1 = pd.read_csv('bundesliga1.csv')

In [33]:
df_ligue11 = pd.read_csv('ligue1.csv')

In [34]:
df_ligue11.head()

,Wk,Day,Date,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee
0,1.0,Fri,8/10/18,Marseille,2.6,4–0,0.3,Toulouse,60756.0,Orange Vélodrome,Ruddy Buquet
1,1.0,Sat,8/11/18,Nantes,1.7,1–3,2.2,Monaco,32760.0,Stade de la Beaujoire - Louis Fonteneau,Jérôme Brisard
2,1.0,Sat,8/11/18,Montpellier,2.0,1–2,1.9,Dijon,12765.0,Stade de la Mosson,Florent Batta
3,1.0,Sat,8/11/18,Angers,2.5,3–4,1.9,Nîmes,9534.0,Stade Raymond Kopa,Antony Gautier
4,1.0,Sat,8/11/18,Saint-Étienne,1.0,2–1,2.0,Guingamp,26006.0,Stade Geoffroy-Guichard,Amaury Delerue


In [48]:
df_ligue11.columns

Index(['Wk', 'Day', 'Date', 'Home', 'xG', 'Score', 'xG.1', 'Away',
       'Attendance', 'Venue', 'Referee'],
      dtype='object')

In [43]:
df_laliga1 = df_laliga1.drop(columns=['Time', 'Match Report', 'Notes'])

In [46]:
df_bundesliga1 = df_bundesliga1.drop(columns=['Time'])

In [49]:
df_epl1['league'] = 'epl'

In [51]:
df_ligue11['league'] = 'ligue 1'

In [52]:
df_bundesliga1['league'] = 'bundesliga'

In [53]:
df_seriea1['league'] = 'serie a'

In [54]:
df_laliga1['league'] = 'la liga'

In [55]:
df_mix1 = pd.concat([df_epl1, df_ligue11])

In [58]:
df_mix2 = pd.concat([df_mix1, df_bundesliga1])

In [59]:
df_mix3 = pd.concat([df_mix2, df_seriea1])

In [60]:
df_xg = pd.concat([df_mix3, df_laliga1])

In [62]:
df_xg.shape

(2020, 12)

In [67]:
df_xg['Score'].str.split('–').str[1]

0      1
1      2
2      3
3      2
4      0
      ..
413    1
414    2
415    1
416    2
417    2
Name: Score, Length: 2020, dtype: object

In [68]:
df_xg['home_goals'] = df_xg['Score'].str.split('–').str[0]

In [69]:
df_xg['away_goals'] = df_xg['Score'].str.split('–').str[1]

In [71]:
df_xg = df_xg.drop(columns=['Score'])

In [77]:
df_xg.isna().sum()

Wk            194
Day           194
Date          194
Home          194
xG            194
xG.1          194
Away          194
Attendance    196
Venue         194
Referee       194
league          0
home_goals    194
away_goals    194
dtype: int64

In [82]:
df_xg = df_xg.dropna()

In [84]:
df_xg.isna().sum()

Wk            0
Day           0
Date          0
Home          0
xG            0
xG.1          0
Away          0
Attendance    0
Venue         0
Referee       0
league        0
home_goals    0
away_goals    0
dtype: int64

In [87]:
df_xg['home_goals'] = df_xg['home_goals'].astype('int32')

/Users/andrewtriola/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
df_xg['away_goals'] = df_xg['away_goals'].astype('int32')

/Users/andrewtriola/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
team_list = (df_xg['Home'].unique())

In [98]:
team_list = team_list.tolist()

In [106]:
df_xg['Home'] = df_xg['Home'].astype('str')

In [110]:
df_xg.dtypes

Wk            float64
Day            object
Date           object
Home           object
xG            float64
xG.1          float64
Away           object
Attendance    float64
Venue          object
Referee        object
league         object
home_goals      int32
away_goals      int32
dtype: object

In [122]:
xg_home_dict = {}
xg_away_dict = {}
xg_total_dict = {}
for team in team_list:
    home_team_df = df_xg[df_xg['Home'] == team]
    xg_home_dict[team] = home_team_df['xG'].sum()
    away_team_df = df_xg[df_xg['Away'] == team]
    xg_away_dict[team] = away_team_df['xG.1'].sum()
    xg_total_dict[team] = xg_home_dict[team] + xg_away_dict[team]

In [130]:
xg_total_dict

{'Manchester Utd': 59.5,
 'Newcastle Utd': 38.800000000000004,
 'Huddersfield': 30.099999999999998,
 'Fulham': 39.400000000000006,
 'Watford': 47.9,
 'Bournemouth': 53.7,
 'Wolves': 49.5,
 'Southampton': 45.2,
 'Liverpool': 73.0,
 'Arsenal': 56.400000000000006,
 'Cardiff City': 39.4,
 'West Ham': 48.8,
 'Everton': 47.5,
 'Leicester City': 48.89999999999999,
 'Tottenham': 54.7,
 'Chelsea': 58.0,
 'Burnley': 41.800000000000004,
 'Manchester City': 81.7,
 'Brighton': 34.9,
 'Crystal Palace': 50.7,
 'Marseille': 49.7,
 'Nantes': 47.400000000000006,
 'Montpellier': 44.49999999999999,
 'Angers': 47.60000000000001,
 'Saint-Étienne': 55.5,
 'Nice': 42.300000000000004,
 'Lille': 56.7,
 'Lyon': 71.8,
 'Bordeaux': 38.7,
 'Paris S-G': 91.1,
 'Reims': 40.0,
 'Guingamp': 40.3,
 'Dijon': 39.2,
 'Rennes': 53.099999999999994,
 'Caen': 31.400000000000002,
 'Amiens': 34.7,
 'Monaco': 48.7,
 'Strasbourg': 41.39999999999999,
 'Toulouse': 43.900000000000006,
 'Nîmes': 46.599999999999994,
 'Bayern Munich': 7

In [121]:
xga_total_dict

{'Manchester Utd': 59.5,
 'Newcastle Utd': 38.800000000000004,
 'Huddersfield': 30.099999999999998,
 'Fulham': 39.400000000000006,
 'Watford': 47.9,
 'Bournemouth': 53.7,
 'Wolves': 49.5,
 'Southampton': 45.2,
 'Liverpool': 73.0,
 'Arsenal': 56.400000000000006,
 'Cardiff City': 39.4,
 'West Ham': 48.8,
 'Everton': 47.5,
 'Leicester City': 48.89999999999999,
 'Tottenham': 54.7,
 'Chelsea': 58.0,
 'Burnley': 41.800000000000004,
 'Manchester City': 81.7,
 'Brighton': 34.9,
 'Crystal Palace': 50.7,
 'Marseille': 49.7,
 'Nantes': 47.400000000000006,
 'Montpellier': 44.49999999999999,
 'Angers': 47.60000000000001,
 'Saint-Étienne': 55.5,
 'Nice': 42.300000000000004,
 'Lille': 56.7,
 'Lyon': 71.8,
 'Bordeaux': 38.7,
 'Paris S-G': 91.1,
 'Reims': 40.0,
 'Guingamp': 40.3,
 'Dijon': 39.2,
 'Rennes': 53.099999999999994,
 'Caen': 31.400000000000002,
 'Amiens': 34.7,
 'Monaco': 48.7,
 'Strasbourg': 41.39999999999999,
 'Toulouse': 43.900000000000006,
 'Nîmes': 46.599999999999994,
 'Bayern Munich': 7

In [127]:
xga_home_dict = {}
xga_away_dict = {}
xga_total_dict = {}
for team in team_list:
    home_team_df = df_xg[df_xg['Home'] == team]
    xga_home_dict[team] = home_team_df['xG.1'].sum()
    away_team_df = df_xg[df_xg['Away'] == team]
    xga_away_dict[team] = away_team_df['xG'].sum()
    xga_total_dict[team] = xga_home_dict[team] + xga_away_dict[team]

In [129]:
xga_total_dict

{'Manchester Utd': 48.300000000000004,
 'Newcastle Utd': 53.39999999999999,
 'Huddersfield': 62.3,
 'Fulham': 65.5,
 'Watford': 55.1,
 'Bournemouth': 54.8,
 'Wolves': 38.8,
 'Southampton': 55.0,
 'Liverpool': 30.099999999999998,
 'Arsenal': 55.7,
 'Cardiff City': 61.9,
 'West Ham': 61.400000000000006,
 'Everton': 44.5,
 'Leicester City': 42.2,
 'Tottenham': 45.599999999999994,
 'Chelsea': 34.6,
 'Burnley': 63.000000000000014,
 'Manchester City': 25.099999999999998,
 'Brighton': 54.900000000000006,
 'Crystal Palace': 47.7,
 'Marseille': 44.9,
 'Nantes': 46.7,
 'Montpellier': 42.3,
 'Angers': 46.5,
 'Saint-Étienne': 50.0,
 'Nice': 48.3,
 'Lille': 43.0,
 'Lyon': 42.5,
 'Bordeaux': 50.7,
 'Paris S-G': 36.1,
 'Reims': 44.300000000000004,
 'Guingamp': 48.5,
 'Dijon': 59.8,
 'Rennes': 47.7,
 'Caen': 57.0,
 'Amiens': 45.599999999999994,
 'Monaco': 48.099999999999994,
 'Strasbourg': 49.699999999999996,
 'Toulouse': 57.599999999999994,
 'Nîmes': 55.3,
 'Bayern Munich': 26.3,
 'Werder Bremen': 54

In [131]:
xg_dif_dict = {}
for team in team_list:
    xg_dif_dict[team] = xg_total_dict[team] - xga_total_dict[team]

In [132]:
xg_dif_dict

{'Manchester Utd': 11.199999999999996,
 'Newcastle Utd': -14.599999999999987,
 'Huddersfield': -32.2,
 'Fulham': -26.099999999999994,
 'Watford': -7.200000000000003,
 'Bournemouth': -1.0999999999999943,
 'Wolves': 10.700000000000003,
 'Southampton': -9.799999999999997,
 'Liverpool': 42.900000000000006,
 'Arsenal': 0.7000000000000028,
 'Cardiff City': -22.5,
 'West Ham': -12.600000000000009,
 'Everton': 3.0,
 'Leicester City': 6.699999999999989,
 'Tottenham': 9.100000000000009,
 'Chelsea': 23.4,
 'Burnley': -21.20000000000001,
 'Manchester City': 56.60000000000001,
 'Brighton': -20.000000000000007,
 'Crystal Palace': 3.0,
 'Marseille': 4.800000000000004,
 'Nantes': 0.7000000000000028,
 'Montpellier': 2.1999999999999957,
 'Angers': 1.1000000000000085,
 'Saint-Étienne': 5.5,
 'Nice': -5.999999999999993,
 'Lille': 13.700000000000003,
 'Lyon': 29.299999999999997,
 'Bordeaux': -12.0,
 'Paris S-G': 54.99999999999999,
 'Reims': -4.300000000000004,
 'Guingamp': -8.200000000000003,
 'Dijon': -20

In [133]:
xgs_list = [xg_home_dict, xg_away_dict, xg_total_dict, xga_home_dict, xga_away_dict, xga_total_dict, xg_dif_dict]

In [138]:
xg_df = pd.DataFrame(xgs_list)

In [139]:
xg_df.head()

,Manchester Utd,Newcastle Utd,Huddersfield,Fulham,Watford,Bournemouth,Wolves,Southampton,Liverpool,Arsenal,...,Getafe,Leganés,Alavés,Atlético Madrid,Valladolid,Espanyol,Sevilla,Levante,Huesca,Real Sociedad
0,31.4,22.1,15.9,22.7,23.1,28.6,29.2,22.8,41.6,32.7,...,26.1,20.8,22.1,24.1,21.9,27.3,38.6,29.0,26.7,27.0
1,28.1,16.7,14.2,16.7,24.8,25.1,20.3,22.4,31.4,23.7,...,18.2,15.8,15.6,20.4,16.8,21.2,22.9,20.5,17.8,18.9
2,59.5,38.8,30.1,39.4,47.9,53.7,49.5,45.2,73.0,56.4,...,44.3,36.6,37.7,44.5,38.7,48.5,61.5,49.5,44.5,45.9
3,20.3,23.7,30.1,27.7,27.5,24.2,17.5,25.0,13.7,24.3,...,17.0,15.8,21.7,16.5,20.3,24.0,18.9,29.8,22.5,17.9
4,28.0,29.7,32.2,37.8,27.6,30.6,21.3,30.0,16.4,31.4,...,22.6,22.8,28.1,21.3,31.9,26.9,24.6,37.7,32.8,24.5


In [140]:
xg_df = xg_df.T

In [142]:
col_list = ['xg_home', 'xg_away', 'xg_total', 'xga_home', 'xga_away', 'xga_total', 'xg_dif']

In [143]:
xg_df.columns = col_list

In [275]:
df_xg.head()

,Wk,Day,Date,Home,xG,xG.1,Away,Attendance,Venue,Referee,league,home_goals,away_goals
0,1.0,Fri,8/10/18,Manchester Utd,1.6,1.1,Leicester City,74439.0,Old Trafford,Andre Marriner,epl,2,1
1,1.0,Sat,8/11/18,Newcastle Utd,0.9,1.9,Tottenham,51749.0,St. James' Park,Martin Atkinson,epl,1,2
2,1.0,Sat,8/11/18,Huddersfield,0.5,1.9,Chelsea,24121.0,The John Smith's Stadium,Chris Kavanagh,epl,0,3
3,1.0,Sat,8/11/18,Fulham,0.6,0.9,Crystal Palace,24821.0,Craven Cottage,Mike Dean,epl,0,2
4,1.0,Sat,8/11/18,Watford,1.6,0.3,Brighton,20051.0,Vicarage Road Stadium,Jonathan Moss,epl,2,0


In [144]:
xg_df.head()

,xg_home,xg_away,xg_total,xga_home,xga_away,xga_total,xg_dif
Manchester Utd,31.4,28.1,59.5,20.3,28.0,48.3,11.2
Newcastle Utd,22.1,16.7,38.8,23.7,29.7,53.4,-14.6
Huddersfield,15.9,14.2,30.1,30.1,32.2,62.3,-32.2
Fulham,22.7,16.7,39.4,27.7,37.8,65.5,-26.1
Watford,23.1,24.8,47.9,27.5,27.6,55.1,-7.2


In [145]:
df_xg.to_csv(r'/Users/andrewtriola/Documents/flatiron/final_project/final_project_repo/df_xg.csv')

In [146]:
xg_df.to_csv(r'/Users/andrewtriola/Documents/flatiron/final_project/final_project_repo/xg_df.csv')

In [147]:
team_list

['Manchester Utd',
 'Newcastle Utd',
 'Huddersfield',
 'Fulham',
 'Watford',
 'Bournemouth',
 'Wolves',
 'Southampton',
 'Liverpool',
 'Arsenal',
 'Cardiff City',
 'West Ham',
 'Everton',
 'Leicester City',
 'Tottenham',
 'Chelsea',
 'Burnley',
 'Manchester City',
 'Brighton',
 'Crystal Palace',
 'Marseille',
 'Nantes',
 'Montpellier',
 'Angers',
 'Saint-Étienne',
 'Nice',
 'Lille',
 'Lyon',
 'Bordeaux',
 'Paris S-G',
 'Reims',
 'Guingamp',
 'Dijon',
 'Rennes',
 'Caen',
 'Amiens',
 'Monaco',
 'Strasbourg',
 'Toulouse',
 'Nîmes',
 'Bayern Munich',
 'Werder Bremen',
 'Freiburg',
 'Wolfsburg',
 'Hertha BSC',
 'Düsseldorf',
 "M'gladbach",
 'Mainz 05',
 'Dortmund',
 'Hannover 96',
 'Augsburg',
 'Hoffenheim',
 'Bayer',
 'Eint Frankfurt',
 'Nürnberg',
 'Stuttgart',
 'RB Leipzig',
 'Schalke 04',
 'Chievo',
 'Lazio',
 'Torino',
 'Empoli',
 'Bologna',
 'Parma',
 'Sassuolo',
 'Atalanta',
 'Juventus',
 'Napoli',
 'SPAL',
 'Inter',
 'Cagliari',
 'Udinese',
 'Fiorentina',
 'Genoa',
 'Roma',
 'Milan'

In [274]:
wiki_list = ['/wiki/Manchester_United_F.C.', '/wiki/Newcastle_United_F.C.', '/wiki/Huddersfield_Town_A.F.C.',
'/wiki/Fulham_F.C.', '/wiki/Watford_F.C.', '/wiki/A.F.C._Bournemouth', '/wiki/Wolverhampton_Wanderers_F.C.',
'/wiki/Southampton_F.C.', '/wiki/Liverpool_F.C.', '/wiki/Arsenal_F.C.', '/wiki/Cardiff_City_F.C.',
'/wiki/West_Ham_United_F.C.', '/wiki/Everton_F.C.', '/wiki/Leicester_City_F.C.', '/wiki/Tottenham_Hotspur_F.C.',
'/wiki/Chelsea_F.C.', '/wiki/Burnley_F.C.', '/wiki/Manchester_City_F.C.', '/wiki/Brighton_%26_Hove_Albion_F.C.',
'/wiki/Crystal_Palace_F.C.', '/wiki/Olympique_de_Marseille', '/wiki/FC_Nantes', '/wiki/Montpellier_HSC',
'/wiki/Angers_SCO', '/wiki/AS_Saint-Étienne', '/wiki/OGC_Nice', '/wiki/Lille_OSC', '/wiki/Olympique_Lyonnais',
'/wiki/FC_Girondins_de_Bordeaux', '/wiki/Paris_Saint-Germain_F.C.', '/wiki/Stade_de_Reims', '/wiki/En_Avant_de_Guingamp',
'/wiki/Dijon_FCO', '/wiki/Stade_Rennais_F.C.', '/wiki/Stade_Malherbe_Caen', '/wiki/Amiens_SC', '/wiki/AS_Monaco_FC',
'/wiki/RC_Strasbourg_Alsace', '/wiki/Toulouse_FC', '/wiki/Nîmes_Olympique', '/wiki/FC_Bayern_Munich', '/wiki/SV_Werder_Bremen',
'/wiki/SC_Freiburg', '/wiki/VfL_Wolfsburg', '/wiki/Hertha_BSC', '/wiki/Fortuna_Düsseldorf',
'/wiki/Borussia_Mönchengladbach', '/wiki/1._FSV_Mainz_05', '/wiki/Borussia_Dortmund', '/wiki/Hannover_96',
'/wiki/FC_Augsburg', '/wiki/TSG_1899_Hoffenheim', '/wiki/Bayer_04_Leverkusen', '/wiki/Eintracht_Frankfurt',
'/wiki/1._FC_Nürnberg', '/wiki/VfB_Stuttgart', '/wiki/RB_Leipzig', '/wiki/FC_Schalke_04', '/wiki/A.C._ChievoVerona',
'/wiki/S.S._Lazio', '/wiki/Torino_F.C.', '/wiki/Empoli_F.C.', '/wiki/Bologna_F.C._1909', '/wiki/Parma_Calcio_1913',
'/wiki/U.S._Sassuolo_Calcio', '/wiki/Atalanta_B.C.', '/wiki/Juventus_F.C.', '/wiki/S.S.C._Napoli', '/wiki/S.P.A.L.',
'/wiki/Inter_Milan', '/wiki/Cagliari_Calcio', '/wiki/Udinese_Calcio', '/wiki/ACF_Fiorentina', '/wiki/Genoa_C.F.C.',
'/wiki/A.S._Roma', '/wiki/A.C._Milan', '/wiki/U.C._Sampdoria', '/wiki/Frosinone_Calcio', '/wiki/Girona_FC',
'/wiki/Real_Betis', '/wiki/RC_Celta_de_Vigo', '/wiki/Villarreal_CF', '/wiki/FC_Barcelona', '/wiki/SD_Eibar',
'/wiki/Rayo_Vallecano', '/wiki/Real_Madrid_CF', '/wiki/Valencia_CF', '/wiki/Athletic_Bilbao', '/wiki/Getafe_CF',
'/wiki/CD_Leganés', '/wiki/Deportivo_Alavés', '/wiki/Atlético_Madrid', '/wiki/Real_Valladolid', '/wiki/RCD_Espanyol',
'/wiki/Sevilla_FC', '/wiki/Levante_UD', '/wiki/SD_Huesca', '/wiki/Real_Sociedad']

In [169]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
wiki_url = 'https://en.wikipedia.org'+team_wiki

In [230]:
page=requests.get('https://en.wikipedia.org/wiki/Real_Sociedad',headers=headers)

In [231]:
page

<Response [200]>

In [232]:
soup = BS(page.content, 'html.parser')
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Real Sociedad - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xkh37gpAICkAAKg81LAAAABY","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Real_Sociedad","wgTitle":"Real Sociedad","wgCurRevisionId":940082523,"wgRevisionId":940082523,"wgArticleId":143279,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Spanish-language sources (es)","Official website d

In [233]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [234]:
html = list(soup.children)[2]
html

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Real Sociedad - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xkh37gpAICkAAKg81LAAAABY","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Real_Sociedad","wgTitle":"Real Sociedad","wgCurRevisionId":940082523,"wgRevisionId":940082523,"wgArticleId":143279,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Spanish-language sources (es)","Official website different in Wikidata and Wikipedia","Articles w

In [235]:
[type(item) for item in list(html.children)]

[bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag]

In [236]:
body = list(html.children)[3]

In [237]:
[type(item) for item in list(body.children)]

[bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString,
 bs4.element.Tag]

In [238]:
para = list(body.children)[5]

In [239]:
para

<div class="mw-body" id="content" role="main">
<a id="top"></a>
<div class="mw-body-content" id="siteNotice"><!-- CentralNotice --></div>
<div class="mw-indicators mw-body-content">
</div>
<h1 class="firstHeading" id="firstHeading" lang="en">Real Sociedad</h1>
<div class="mw-body-content" id="bodyContent">
<div class="noprint" id="siteSub">From Wikipedia, the free encyclopedia</div>
<div id="contentSub"></div>
<div id="jump-to-nav"></div>
<a class="mw-jump-link" href="#mw-head">Jump to navigation</a>
<a class="mw-jump-link" href="#p-search">Jump to search</a>
<div class="mw-content-ltr" dir="ltr" id="mw-content-text" lang="en"><div class="mw-parser-output"><div class="hatnote navigation-not-searchable" role="note">Not to be confused with <a href="/wiki/C.D._Real_Sociedad" title="C.D. Real Sociedad">C.D. Real Sociedad</a>.</div>
<table class="infobox vcard" style="width:22em"><caption class="fn org">Real Sociedad</caption><tbody><tr><td colspan="2" style="text-align:center"><a class="im

In [240]:
text = para.find_all('p')


In [255]:
team_team=[]
team_str = ""
for node in text:
    team_team.append("".join(node.findAll(text=True)))
team_str = team_str.join(team_team)

In [256]:
team_str

'Real Sociedad de Fútbol, S.A.D., more commonly referred to as Real Sociedad (pronounced\xa0[reˈal soθjeˈðað]; Royal Society) or La Real, is a Spanish sports club in the city of San Sebastián, Basque Country, founded on 7 September 1909. It plays its home matches at the Anoeta Stadium. Real Sociedad won the Liga title in 1980–81 and 1981–82, and last finished runner-up in 2002–03. The club has also won the Copa del Rey twice, in 1909 and 1987. It contests the Basque derby against rivals Athletic Bilbao. Real Sociedad was a founder member of La Liga in 1929; its longest spell in the top flight was for 40 seasons, from 1967 to 2007.[2]\nTraditionally the club followed a policy (similar to that of its rival Athletic) of signing only Basque players, before signing Republic of Ireland forward John Aldridge in 1989. While a strong Basque contingent has been retained among its players, nowadays both non-Basque Spaniards and foreign players are represented at the club. Its youth section subseq

In [370]:
def get_team_wiki(team_wiki):
    start_time = time.time()
    team_team = []
    team_str = ""
    wiki_url = 'https://en.wikipedia.org'+team_wiki    
    page=requests.get(wiki_url,headers=headers)    
    soup = BS(page.content, 'html.parser')
    html = list(soup.children)[2]
    body = list(html.children)[3]
    para = list(body.children)[5]
    text = para.find_all('p')
    for node in text:
        team_team.append("".join(node.findAll(text=True)))
    team_str = team_str.join(team_team)
    team_str1 = team_str.replace('\n','')
    team_str2 = team_str1.replace("\xa0","")
    team_str3 = team_str2.replace('\\','')
    print("--- %s seconds ---" % (time.time() - start_time))
    return team_str3

In [371]:
lfc_str = get_team_wiki('/wiki/Liverpool_F.C.')

--- 1.2727670669555664 seconds ---


In [372]:
lfc_str

'Liverpool Football Club is a professional football club in Liverpool, England, that competes in the Premier League, the top tier of English football. The club has won six European Cups, more than any other English club, three UEFA Cups, four UEFA Super Cups (both also English records), one FIFA Club World Cup, eighteen League titles, seven FA Cups, a record eight League Cups, fifteen FA Community Shields and one Football League Super Cup.Founded in 1892, the club joined the Football League the following year and has played at Anfield since its formation. Liverpool established itself as a major force in English and European football in the 1970s and 1980s when Bill Shankly, Bob Paisley, Joe Fagan and Kenny Dalglish led the club to a combined eleven League titles and four European Cups. Under the management of Rafael Benítez and captained by homegrown player Steven Gerrard, Liverpool became European champions for the fifth time in 2005, before a sixth title was added under Jürgen Klopp 

In [266]:
team_wiki_list1 = []
for item in wiki_list:
    wiki_sum = get_team_wiki(item)
    team_wiki_list1.append(wiki_sum)

--- 0.9940111637115479 seconds ---
--- 0.6869840621948242 seconds ---
--- 0.5656776428222656 seconds ---
--- 0.5659382343292236 seconds ---
--- 0.5579540729522705 seconds ---
--- 0.4434378147125244 seconds ---
--- 0.5823490619659424 seconds ---
--- 0.6394810676574707 seconds ---
--- 1.5624117851257324 seconds ---
--- 1.8842277526855469 seconds ---
--- 2.4979851245880127 seconds ---
--- 1.0268089771270752 seconds ---
--- 0.755681037902832 seconds ---
--- 0.895514965057373 seconds ---
--- 1.069237232208252 seconds ---
--- 1.4967830181121826 seconds ---
--- 1.1151549816131592 seconds ---
--- 2.4846458435058594 seconds ---
--- 0.9803810119628906 seconds ---
--- 0.8072750568389893 seconds ---
--- 0.653723955154419 seconds ---
--- 0.739271879196167 seconds ---
--- 0.5433948040008545 seconds ---
--- 0.332247257232666 seconds ---
--- 0.4125649929046631 seconds ---
--- 0.43329310417175293 seconds ---
--- 0.4624748229980469 seconds ---
--- 0.6236460208892822 seconds ---
--- 0.48922014236450195 s

In [325]:
def replace_items_1(team_list):
    start_time = time.time()
    team_list2 = []
    for item in team_list:
        item1 = item.replace("\xa0","")
        team_list2.append(item1)
    print("--- %s seconds ---" % (time.time() - start_time))
    return team_list2

In [326]:
team_wiki_list2 = replace_items_1(team_wiki_list1)

--- 0.029964685440063477 seconds ---


In [347]:
team_wiki_list2[0]

'Manchester United Football Club is a professional football club based in Old Trafford, Greater Manchester, England, that competes in the Premier League, the top flight of English football. Nicknamed "the Red Devils", the club was founded as Newton Heath LYR Football Club in 1878, changed its name to Manchester United in 1902 and moved to its current stadium, Old Trafford, in 1910.Manchester United have won more trophies than any other club in English football,[5][6] with a record 20 League titles, 12 FA Cups, five League Cups and a record 21 FA Community Shields. United have also won three UEFA Champions Leagues, one UEFA Europa League, one UEFA Cup Winners\' Cup, one UEFA Super Cup, one Intercontinental Cup and one FIFA Club World Cup. In 1998–99, the club became the first in the history of English football to achieve the continental European treble.[7] By winning the UEFA Europa League in 2016–17, they became one of five clubs to have won all three main UEFA club competitions.The 19

In [384]:
team_wiki_list4 = []
for team in team_wiki_list3:
    for symbol in "[,.?!''\n":
        team = team.replace(symbol, '').lower()
        team_wiki_list4.append(team)

In [389]:
len(team_wiki_list2)

98

In [341]:
def replace_items_2(team_list):
    start_time = time.time()
    team_list2 = []
    for item in team_list:
        item1 = item.replace("\'","'")
        team_list2.append(item1)
    print("--- %s seconds ---" % (time.time() - start_time))
    return team_list2

In [342]:
# team_wiki_list3 = replace_items_2(team_wiki_list2)

--- 2.5221750736236572 seconds ---


In [343]:
# team_wiki_list3[0]

'Manchester United Football Club is a professional football club based in Old Trafford, Greater Manchester, England, that competes in the Premier League, the top flight of English football. Nicknamed "the Red Devils", the club was founded as Newton Heath LYR Football Club in 1878, changed its name to Manchester United in 1902 and moved to its current stadium, Old Trafford, in 1910.Manchester United have won more trophies than any other club in English football,[5][6] with a record 20 League titles, 12 FA Cups, five League Cups and a record 21 FA Community Shields. United have also won three UEFA Champions Leagues, one UEFA Europa League, one UEFA Cup Winners\' Cup, one UEFA Super Cup, one Intercontinental Cup and one FIFA Club World Cup. In 1998–99, the club became the first in the history of English football to achieve the continental European treble.[7] By winning the UEFA Europa League in 2016–17, they became one of five clubs to have won all three main UEFA club competitions.The 19

In [390]:
wiki_dict = {'team': team_list, 'description': team_wiki_list2}

In [391]:
wiki_df = pd.DataFrame(wiki_dict)

In [350]:
wiki_df.head(20)

,team,description
0,Manchester Utd,Manchester United Football Club is a professio...
1,Newcastle Utd,Newcastle United Football Club is an English p...
2,Huddersfield,Huddersfield Town Association Football Club is...
3,Fulham,Fulham Football Club is a professional associa...
4,Watford,Watford Football Club is an English profession...
5,Bournemouth,Athletic Football Club Bournemouth (/ˈbɔːrnməθ...
6,Wolves,Wolverhampton Wanderers Football Club /ˌwʊlvər...
7,Southampton,Southampton Football Club (/saʊθˈ(h)æmptən/ (l...
8,Liverpool,Liverpool Football Club is a professional foot...
9,Arsenal,Arsenal Football Club is a professional footba...


In [365]:
wiki2 = wiki_df.replace({'description': {'[': '', ']': '', '\\': ''}})

In [366]:
wiki2['description'][0]

'Manchester United Football Club is a professional football club based in Old Trafford, Greater Manchester, England, that competes in the Premier League, the top flight of English football. Nicknamed "the Red Devils", the club was founded as Newton Heath LYR Football Club in 1878, changed its name to Manchester United in 1902 and moved to its current stadium, Old Trafford, in 1910.Manchester United have won more trophies than any other club in English football,[5][6] with a record 20 League titles, 12 FA Cups, five League Cups and a record 21 FA Community Shields. United have also won three UEFA Champions Leagues, one UEFA Europa League, one UEFA Cup Winners\' Cup, one UEFA Super Cup, one Intercontinental Cup and one FIFA Club World Cup. In 1998–99, the club became the first in the history of English football to achieve the continental European treble.[7] By winning the UEFA Europa League in 2016–17, they became one of five clubs to have won all three main UEFA club competitions.The 19

In [276]:
df_xg.head()

,Wk,Day,Date,Home,xG,xG.1,Away,Attendance,Venue,Referee,league,home_goals,away_goals
0,1.0,Fri,8/10/18,Manchester Utd,1.6,1.1,Leicester City,74439.0,Old Trafford,Andre Marriner,epl,2,1
1,1.0,Sat,8/11/18,Newcastle Utd,0.9,1.9,Tottenham,51749.0,St. James' Park,Martin Atkinson,epl,1,2
2,1.0,Sat,8/11/18,Huddersfield,0.5,1.9,Chelsea,24121.0,The John Smith's Stadium,Chris Kavanagh,epl,0,3
3,1.0,Sat,8/11/18,Fulham,0.6,0.9,Crystal Palace,24821.0,Craven Cottage,Mike Dean,epl,0,2
4,1.0,Sat,8/11/18,Watford,1.6,0.3,Brighton,20051.0,Vicarage Road Stadium,Jonathan Moss,epl,2,0


In [286]:
league_df = df_xg[['Home','league']]

In [288]:
league_df.drop_duplicates(subset ="Home", keep = 'first', inplace = True)

/Users/andrewtriola/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [290]:
league_df.shape

(98, 2)

In [351]:
merged_df = pd.merge(league_df, wiki_df, how='inner', left_on='Home', right_on='team')


In [352]:
merged_df.head()

,Home,league,team,description
0,Manchester Utd,epl,Manchester Utd,Manchester United Football Club is a professio...
1,Newcastle Utd,epl,Newcastle Utd,Newcastle United Football Club is an English p...
2,Huddersfield,epl,Huddersfield,Huddersfield Town Association Football Club is...
3,Fulham,epl,Fulham,Fulham Football Club is a professional associa...
4,Watford,epl,Watford,Watford Football Club is an English profession...


In [353]:
merged_df = merged_df.drop(columns='Home')

In [354]:
merged_df.shape

(98, 3)

In [367]:
merged_df.to_csv(r'/Users/andrewtriola/Documents/flatiron/final_project/final_project_repo/merged_df.csv')

In [307]:
df_stats.to_csv(r'/Users/andrewtriola/Documents/flatiron/final_project/final_project_repo/df_stats.csv')